In [4]:
import requests
import json
base_url = "https://aqs.epa.gov/data/api"
key = "aquacrane72"
email = "jonathonh.davis@gmail.com"

In [5]:
def get_signup(email):
    url = f"{base_url}/signup?email={email}"
    response = requests.get(url)
    if response.status_code == 200:
        data = json.loads(response.text)
        return data
    else:
        print(f"Error: {response.status_code}")
    return

In [6]:
# Uncomment to for first time use then comment out again
# get_signup(email)

In [7]:
# Get a list of all the States FIPs codes
response = requests.get('https://aqs.epa.gov/data/api/list/states?email=test@aqs.api&key=test')
# print(response.text)
data  = json.loads(response.text)
print(data)


{'Header': [{'status': 'Success', 'request_time': '2024-10-06T06:37:13-04:00', 'url': 'https://aqs.epa.gov/data/api/list/states?email=test@aqs.api&key=test', 'rows': 56}], 'Data': [{'code': '01', 'value_represented': 'Alabama'}, {'code': '02', 'value_represented': 'Alaska'}, {'code': '04', 'value_represented': 'Arizona'}, {'code': '05', 'value_represented': 'Arkansas'}, {'code': '06', 'value_represented': 'California'}, {'code': '08', 'value_represented': 'Colorado'}, {'code': '09', 'value_represented': 'Connecticut'}, {'code': '10', 'value_represented': 'Delaware'}, {'code': '11', 'value_represented': 'District Of Columbia'}, {'code': '12', 'value_represented': 'Florida'}, {'code': '13', 'value_represented': 'Georgia'}, {'code': '15', 'value_represented': 'Hawaii'}, {'code': '16', 'value_represented': 'Idaho'}, {'code': '17', 'value_represented': 'Illinois'}, {'code': '18', 'value_represented': 'Indiana'}, {'code': '19', 'value_represented': 'Iowa'}, {'code': '20', 'value_represented'

In [8]:
# Dictionary mapping state abbreviations to full names
state_abbreviations = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 
    'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 
    'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 
    'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 
    'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 
    'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 
    'WI': 'Wisconsin', 'WY': 'Wyoming'
}

# Function to convert state abbreviation to full name
def get_state_name(abbreviation):
    return state_abbreviations.get(abbreviation.upper(), "Invalid abbreviation")

#Example usage
# print(get_state_name('TX'))  # Output: Texas
# print(get_state_name('Ca'))  # Output: California
# print(get_state_name('MI'))  # Output: Invalid abbreviation

# List of state abbreviations -> full names using the function for the top 9 states US by population
states = ['TX','OK','NY','MN','MI','PA','FL','IL','WI'] # Modify this list as needed
state_names = []
for i in states:
    state_names.append(get_state_name(i))
print(state_names)


['Texas', 'Oklahoma', 'New York', 'Minnesota', 'Michigan', 'Pennsylvania', 'Florida', 'Illinois', 'Wisconsin']


In [9]:
import pandas as pd

filepath = 'saved_data\cleaned_merged.csv'
# Convert the CSV file into a DataFrame
df = pd.read_csv(filepath)

# Define your predetermined list of states
allowed_states = states  

# Filter the DataFrame to keep only rows where State/Province is in allowed_states
filtered_df = df[df['State/Province'].isin(allowed_states)]

# save the filtered data to a new csv file
filtered_df.to_csv('saved_data/filtered_data_2.csv', index=False)


# Display the filtered DataFrame
print("Filtered DataFrame:")
print(filtered_df)

<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
C:\Users\jonat\AppData\Local\Temp\ipykernel_25448\1801072052.py:3: SyntaxWarning: invalid escape sequence '\c'
  filepath = 'saved_data\cleaned_merged.csv'


Filtered DataFrame:
        Unnamed: 0      Date              Town State/Province  Latitude  \
0                0  12/31/18        Homer City             PA      40.5   
2                2  12/30/18     Crawfordville             FL      30.1   
5                5  12/27/18    Corpus Christi             TX      27.7   
7                7  12/27/18          Pearland             TX      29.5   
8                8  12/27/18           Arcadia             FL      27.3   
...            ...       ...               ...            ...       ...   
111039      112858    1/4/17  Fernandina Beach             FL      30.6   
111040      112859    1/2/17   Saint Augustine             FL      29.7   
111041      112860    1/1/17            Spring             TX      30.0   
111042      112861    1/1/17       San Antonio             TX      29.5   
111043      112862    1/1/17       San Antonio             TX      29.5   

        Longitude seasonal  
0           -79.2     fall  
2           -84.3    

In [10]:
# filepath = 'saved_data/filtered_data.csv'
# filepath = 'saved_data/filtered_data_2.csv'

filepath = 'saved_data/updated_dataset.csv'

# Convert the CSV file into a DataFrame
df = pd.read_csv(filepath)

# using the long and lat to get the nearest county
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time

def get_location_info(latitude, longitude):
    geolocator = Nominatim(user_agent="my_agent")
    tries = 0
    while tries < 3:
        try:
            # Ensure the coordinates are floats and in the correct format
            lat = float(latitude)
            lon = float(longitude)
            location = geolocator.reverse(f"{lat}, {lon}")
            if location:
                address = location.raw['address']
                country = address.get('country', 'Unknown')
                county = address.get('county', 'Unknown')
                return country, county
            else:
                return 'Unknown', 'Unknown'
        except (GeocoderTimedOut, GeocoderServiceError):
            tries += 1
            time.sleep(1)
        except ValueError:
            print(f"Invalid coordinates: Latitude={latitude}, Longitude={longitude}")
            return 'Invalid', 'Invalid'
    return 'Unknown', 'Unknown'

# Load the dataset
df = pd.read_csv(filepath)

# Create new columns for country and county
df['Country'] = ''
df['County'] = ''

# Iterate through the DataFrame and update country and county
for index, row in df.iterrows():
    try:
        # check if conunty and country are already in the dataset
        if row['Country'] != '' and row['County'] != '':
            continue

        latitude = float(row['Latitude'])
        longitude = float(row['Longitude'])
        country, county = get_location_info(latitude, longitude)
        df.at[index, 'Country'] = country
        df.at[index, 'County'] = county
    except ValueError:
        print(f"Invalid data in row {index}: Latitude={row['Latitude']}, Longitude={row['Longitude']}")
        df.at[index, 'Country'] = 'Invalid'
        df.at[index, 'County'] = 'Invalid'

    # Print progress
    if index % 10 == 0:
        print(f"Processed {index} rows")
        # Save the updated DataFrame to a new CSV file
        df.to_csv('saved_data/updated_dataset.csv', index=False)


# save df to a new csv file 'updated_dataset.csv'
df.to_csv('saved_data/updated_dataset.csv', index=False)
print("Processing complete. Updated dataset saved as 'updated_dataset.csv'")
df



Processed 0 rows
Processed 10 rows
Processed 20 rows
Processed 30 rows
Processed 40 rows
Processed 50 rows
Processed 60 rows
Processed 70 rows
Processed 80 rows
Processed 90 rows
Processed 100 rows
Processed 110 rows
Processed 120 rows
Processed 130 rows
Processed 140 rows
Processed 150 rows
Processed 160 rows
Processed 170 rows
Processed 180 rows
Processed 190 rows
Processed 200 rows
Processed 210 rows
Processed 220 rows
Processed 230 rows
Processed 240 rows
Processed 250 rows
Processed 260 rows
Processed 270 rows
Processed 280 rows
Processed 290 rows
Processed 300 rows
Processed 310 rows
Processed 320 rows
Processed 330 rows
Processed 340 rows
Processed 350 rows
Processed 360 rows
Processed 370 rows
Processed 380 rows
Processed 390 rows
Processed 400 rows
Processed 410 rows
Processed 420 rows
Processed 430 rows
Processed 440 rows
Processed 450 rows
Processed 460 rows
Processed 470 rows
Processed 480 rows
Processed 490 rows
Processed 500 rows
Processed 510 rows
Processed 520 rows
Proc

In [85]:
# save df to a new csv file 'updated_dataset.csv'
df.to_csv('saved_data/updated_dataset.csv', index=False)
df

,Unnamed: 0,Date,Town,State/Province,Latitude,Longitude,seasonal
0,0,12/31/18,Homer City,PA,40.5,-79.2,fall
1,1,12/30/18,Edisto Island,SC,32.5,-80.3,fall
2,2,12/30/18,Crawfordville,FL,30.1,-84.3,fall
3,3,12/29/18,Edisto Island,SC,32.6,-80.3,fall
4,4,12/28/18,Nags Head,NC,35.8,-75.6,fall
...,...,...,...,...,...,...,...
111047,112867,1/1/17,Pacific Grove,CA,36.6,-121.9,spring
111048,112868,1/1/17,Morro Bay,CA,35.4,-120.8,spring
111049,112869,1/1/17,Santa Cruz,CA,37.1,-122.2,spring
111050,112870,1/1/17,Alder Rd vortex,CA,37.9,-122.7,spring


In [79]:

# Extract the list of state codes for the given list_of_states
list_of_states = ['Florida', 'Texas', 'New York', 'Michigan',]
state_codes = [state['code'] for state in data if state['value_represented'] in list_of_states]
print(state_codes)

# Get the list of counties for the given state codes
county_codes = []


TypeError: string indices must be integers, not 'str'

In [41]:
# Get a list of all the Counties in a State
state_list = ('12', '48', '36', '26' ) # Florida, Texas, New York, Michigan
state_list = ('4')